In [1]:
import pickle
import pandas as pd
from datetime import timedelta
import numpy as np

In [73]:
year=2030
nuc="nuclear_plus"
bati_hyp="ref"
reindus="reindus"
with open("Results_"+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus+".pickle", 'rb') as f:
    Variables=pickle.load(f)

In [74]:
print(Variables)

{'power_Dvar':                       TIME   TECHNOLOGIES    power_Dvar
0      2019-02-28 02:00:00        OldNuke  47720.000000
1      2019-02-28 02:00:00  NewHydroRiver     51.510104
2      2019-02-28 02:00:00            CCG      0.000000
3      2019-02-28 02:00:00   WindOffShore   2665.820000
4      2019-02-28 02:00:00          Solar      0.000000
...                    ...            ...           ...
113875 2019-02-09 04:00:00        NewNuke    792.000000
113876 2019-02-09 04:00:00     HydroRiver   5314.582954
113877 2019-02-09 04:00:00    WindOnShore   3781.440000
113878 2019-02-09 04:00:00        Biomass      0.000000
113879 2019-02-09 04:00:00    curtailment      0.000000

[113880 rows x 3 columns], 'energy_Pvar':                     TIME   energy_Pvar
0    2019-02-28 02:00:00  58985.859394
1    2019-07-07 08:00:00  40120.687216
2    2019-01-31 02:00:00  73843.107587
3    2019-07-29 10:00:00  52473.832361
4    2019-03-15 18:00:00  69015.956999
...                  ...           .

# 1. Capacity 

In [75]:
# Production means capacity (MW)
Variables['capacity_Dvar']

,TECHNOLOGIES,capacity_Dvar
0,OldNuke,59430.000000
1,NewHydroRiver,102.649223
2,CCG,12664.000000
3,WindOffShore,8200.000000
4,Solar,40600.000000
5,Coal,0.000000
6,TAC,0.000000
7,HydroReservoir,8785.000000
8,NewNuke,1650.000000
9,HydroRiver,11030.000000


In [76]:
# Storage capacity
Variables['Pmax_in_Dvar']

,STOCK_TECHNO,Pmax_in_Dvar
0,Battery,9212.356899
1,HydroStorage,5049.000000
2,PowertoH2toPower,6537.562554


In [77]:
# Storage capacity
Variables['Pmax_out_Dvar']

,STOCK_TECHNO,Pmax_out_Dvar
0,Battery,9212.356899
1,HydroStorage,5049.000000
2,PowertoH2toPower,9546.599866


In [78]:
# Storage capacity
Variables['Cmax_Pvar']

,STOCK_TECHNO,Cmax_Pvar
0,Battery,2.399352e+04
1,HydroStorage,9.088200e+04
2,PowertoH2toPower,2.048751e+06


In [79]:
Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()

106414.29031587472

In [80]:
2.399352e+04/(9212.356899*0.95)

2.741571686703414

In [73]:
# Storage means capacity (MW)
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar          138364.610149
dtype: object

In [74]:
Variables['energy_Pvar'].max()

TIME           2019-12-31 23:00:00
energy_Pvar          138364.610149
dtype: object

In [7]:
df=Variables['power_Dvar']
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')]['power_Dvar'].sum())
df2=Variables['storageOut_Pvar']
print(df2[df2.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df3=Variables['storageIn_Pvar']
print(df3[df3.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df4=Variables['stockLevel_Pvar']
print(df4[df4.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(Variables['stockLevel_ini_Dvar'])

                     TIME    TECHNOLOGIES    power_Dvar
40989 2019-12-31 23:00:00     WindOnShore   4432.600000
40990 2019-12-31 23:00:00             CCG      0.000000
40991 2019-12-31 23:00:00         OldNuke  15520.000000
40992 2019-12-31 23:00:00             TAC      0.000000
40993 2019-12-31 23:00:00    WindOffShore  10598.700000
40994 2019-12-31 23:00:00  HydroReservoir   8785.000000
40995 2019-12-31 23:00:00         Biomass   3000.000000
40996 2019-12-31 23:00:00            Coal      0.000000
40997 2019-12-31 23:00:00           Solar      0.000000
40998 2019-12-31 23:00:00         NewNuke  21710.000000
40999 2019-12-31 23:00:00      HydroRiver   8233.891861
41000 2019-12-31 23:00:00   NewHydroRiver    895.799659
41001 2019-12-31 23:00:00     curtailment      0.000000
73175.99151963575
                    TIME      STOCK_TECHNO  storageOut_Pvar
9459 2019-12-31 23:00:00           Battery              0.0
9460 2019-12-31 23:00:00  PowertoH2toPower              0.0
9461 2019-12-31 23

In [8]:
df=Variables['power_Dvar']
df2=Variables['storageOut_Pvar']
df3=Variables['storageIn_Pvar']
df=df.groupby('TIME')['power_Dvar'].sum()
df2=df2.groupby('TIME')['storageOut_Pvar'].sum()
df3=df3.groupby('TIME')['storageIn_Pvar'].sum()
df_prod=pd.DataFrame()
df_prod['Prod']=df+df2-df3
#df_prod
#df_prod=df_prod.set_index('TIME')

df_diff=pd.DataFrame()
df_diff['diff']=df_prod['Prod']-Variables['total_consumption_Pvar'].set_index('TIME')['total_consumption_Pvar']
print(df_diff['diff'].max())
N_err=0
for i in df_diff.index:
    if np.abs(df_diff.loc[i,'diff'])>1:
        N_err+=1
        print(df_diff.loc[i,'diff'])
print(N_err)

4.3655745685100555e-11
0


In [9]:
df_prod.loc[pd.Timestamp('2019-12-31 23:00:00')]

Prod    73175.99152
Name: 2019-12-31 23:00:00, dtype: float64

In [10]:
Variables['total_consumption_Pvar'].set_index('TIME').loc[pd.Timestamp('2019-12-31 23:00:00')]

total_consumption_Pvar    73175.99152
Name: 2019-12-31 23:00:00, dtype: float64

In [21]:
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar            155289.7205
dtype: object

In [12]:
Variables['total_consumption_Pvar'][Variables['total_consumption_Pvar'].total_consumption_Pvar==Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()]

,TIME,total_consumption_Pvar
1290,2019-01-24 13:00:00,150448.942674


In [13]:
d0=pd.Timestamp('2019-01-01 00:00:00')
d1h=timedelta(hours=1)
df=Variables['flex_consumption_Pvar']
L_EV=[]
for h in range(24):
    L_EV.append(float(df[(df.TIME==d0+h*d1h)&(df.FLEX_CONSUM=='EV')]['flex_consumption_Pvar']))
L_EV

[8974.985765378075,
 11252.569669328535,
 10691.645219955992,
 11170.426644899537,
 13510.099229131103,
 15473.200406998334,
 16550.885092623164,
 4425.99954355268,
 5581.337750877695,
 20919.870411678494,
 21837.125896171616,
 23312.874833511825,
 28469.2682218418,
 25998.15287218346,
 27305.36161875611,
 25600.720420410304,
 20896.11392020079,
 18139.698630793904,
 17262.300474917676,
 15669.700657177991,
 15415.259850726845,
 17359.40017950799,
 25017.329324809274,
 23867.510910719506]

In [32]:
def Total_cost(Variables):
    return 1/1e9*(Variables['powerCosts_Pvar']['powerCosts_Pvar'].sum()\
            +Variables['capacityCosts_Pvar']['capacityCosts_Pvar'].sum()\
            +Variables['storageCosts_Pvar']['storageCosts_Pvar'].sum()\
            +float(Variables['flexCosts_Pvar']['flexCosts_Pvar'].sum()))
Total_cost(Variables)

50.9856406845131